## Resolve dependências

In [1]:
!pip install -q albumentations==1.4.6 opencv-python pycocotools tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q pycocotools albumentations==1.4.6 opencv-python tqdm
!pip uninstall -y sympy
!pip install sympy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 9.1 MB/s eta 0:00:00
Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.9 MB/s eta 0:00:00


In [2]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.24.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 694356, done.
remote: Counting objects: 100% (1691/1691), done.
remote: Compressing objects: 100% (879/879), done.
remote: Total 694356 (delta 1519), reused 885 (delta 804), pack-reused 692665 (from 3)
Receiving objects: 100% (694356/694356), 1.22 GiB | 44.46 MiB/s, done.
Resolving deltas: 100% (650252/650252), done.
fatal: not a git repository (or any of the parent directories): .git


In [3]:
import sys
# sys.path.append("/content/drive/MyDrive/TCC/src/")
sys.path.append("/content/drive/MyDrive/TCC/notebooks/Faster R-CNN")

In [4]:
from pathlib import Path

DATASET_ROOT = Path("/content/drive/MyDrive/TCC/DATASET")
paths = {
    "bh_watertanks": {
        "images": {
            "train": DATASET_ROOT/"bh_watertanks/images/train",
            "val":   DATASET_ROOT/"bh_watertanks/images/val",
            "test":  DATASET_ROOT/"bh_watertanks/images/test",
        },
        "ann": {
            "train": DATASET_ROOT/"bh_watertanks/annotations/instances_train.json",
            "val":   DATASET_ROOT/"bh_watertanks/annotations/instances_val.json",
            "test":  DATASET_ROOT/"bh_watertanks/annotations/instances_test.json",
        }
    }
}

In [5]:
import sys
from pathlib import Path
sys.path.append("/content/drive/MyDrive/TCC/src/")
DATASET_ROOT = Path("/content/drive/MyDrive/TCC/campinas_dataset")

campinas_paths = {
    "bh_watertanks": {
        "images": {
            "train": DATASET_ROOT/"train/images/",
            "test":  DATASET_ROOT/"test/images/",
        },
        "ann": {
            "train": DATASET_ROOT/"train/annotations/instances_train.json",
            "test":  DATASET_ROOT/"test/annotations/instances_test.json",
        }
    }
}

## Carrega Loaders

In [12]:
from my_utils.coco_dataset import CocoDetDataset
from my_utils.my_utils import get_transforms, collate_fn, evaluate_and_visualize
from torch.utils.data import Dataset, DataLoader


watertanks_train = CocoDetDataset(
    paths["bh_watertanks"]["images"]["train"],
    paths["bh_watertanks"]["ann"]["train"],
    transforms=get_transforms(train=True))

watertanks_val   = CocoDetDataset(
    paths["bh_watertanks"]["images"]["val"],
    paths["bh_watertanks"]["ann"]["val"],
    transforms=get_transforms(train=False))

watertanks_test  = CocoDetDataset(
    paths["bh_watertanks"]["images"]["test"],
    paths["bh_watertanks"]["ann"]["test"],
    transforms=get_transforms(train=False))


watertanks_train_loader = DataLoader(watertanks_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
watertanks_val_loader   = DataLoader(watertanks_val,   batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
watertanks_test_loader  = DataLoader(watertanks_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

loading annotations into memory...
Done (t=1.32s)
creating index...
index created!
loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
loading annotations into memory...
Done (t=0.53s)
creating index...
index created!


In [6]:
from my_utils.coco_dataset import CocoDetDataset
from my_utils.my_utils import get_transforms, collate_fn, evaluate_and_visualize
from torch.utils.data import Dataset, DataLoader


cmp_watertanks_train = CocoDetDataset(
    campinas_paths["bh_watertanks"]["images"]["train"],
    campinas_paths["bh_watertanks"]["ann"]["train"],
    transforms=get_transforms(train=True))

cmp_watertanks_test  = CocoDetDataset(
    campinas_paths["bh_watertanks"]["images"]["test"],
    campinas_paths["bh_watertanks"]["ann"]["test"],
    transforms=get_transforms(train=False))


cmp_watertanks_train_loader = DataLoader(cmp_watertanks_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
cmp_watertanks_test_loader  = DataLoader(cmp_watertanks_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

loading annotations into memory...
Done (t=0.99s)
creating index...
index created!
loading annotations into memory...
Done (t=1.00s)
creating index...
index created!


## Inicializa model

In [7]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 1 + 2

weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth


100%|██████████| 167M/167M [00:00<00:00, 237MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [9]:
checkpoint_path = '/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/model/wt_t_.pth'
state = torch.load(checkpoint_path)
model.load_state_dict(state)

<All keys matched successfully>

## Treina modelo

In [10]:
from torch.cuda.amp import GradScaler
scaler = GradScaler()

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.002,
    momentum=0.9,
    weight_decay=5e-4
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=10,
    eta_min=1e-5
)


/tmp/ipython-input-900805923.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [11]:
from engine import train_one_epoch
import torch

epochs = 10

for epoch in range(epochs):
  for p in model.backbone.body.parameters():
    p.requires_grad = True
    new_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(new_params, lr=0.002, momentum=0.9, weight_decay=5e-4)

  train_one_epoch(model, optimizer, cmp_watertanks_train_loader, device, epoch, print_freq=10)
  lr_scheduler.step()



/content/engine.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [ 0/51]  eta: 0:03:16  lr: 0.000042  loss: 0.5330 (0.5330)  loss_classifier: 0.2900 (0.2900)  loss_box_reg: 0.0046 (0.0046)  loss_objectness: 0.2261 (0.2261)  loss_rpn_box_reg: 0.0123 (0.0123)  time: 3.8482  data: 1.0665  max mem: 3727
Epoch: [0]  [10/51]  eta: 0:00:28  lr: 0.000442  loss: 0.5003 (0.4853)  loss_classifier: 0.2235 (0.2335)  loss_box_reg: 0.0032 (0.0042)  loss_objectness: 0.2261 (0.2316)  loss_rpn_box_reg: 0.0073 (0.0160)  time: 0.6841  data: 0.1940  max mem: 3913
Epoch: [0]  [20/51]  eta: 0:00:19  lr: 0.000841  loss: 0.3572 (0.3802)  loss_classifier: 0.1227 (0.1467)  loss_box_reg: 0.0032 (0.0071)  loss_objectness: 0.2089 (0.2110)  loss_rpn_box_reg: 0.0044 (0.0155)  time: 0.4762  data: 0.2155  max mem: 3913
Epoch: [0]  [30/51]  eta: 0:00:12  lr: 0.001241  loss: 0.1976 (0.3089)  loss_classifier: 0.0391 (0.1133)  loss_box_reg: 0.0122 (0.0112)  loss_objectness: 0.1204 (0.1709)  loss_rpn_box_reg: 0.0032 (0.0135)  time: 0.5222  data: 0.2613  max mem: 3913
Epoch: [

/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch: [1]  [ 0/51]  eta: 0:00:20  lr: 0.002000  loss: 0.1842 (0.1842)  loss_classifier: 0.0476 (0.0476)  loss_box_reg: 0.0981 (0.0981)  loss_objectness: 0.0288 (0.0288)  loss_rpn_box_reg: 0.0097 (0.0097)  time: 0.3944  data: 0.1308  max mem: 3913
Epoch: [1]  [10/51]  eta: 0:00:11  lr: 0.002000  loss: 0.1397 (0.1500)  loss_classifier: 0.0370 (0.0379)  loss_box_reg: 0.0823 (0.0890)  loss_objectness: 0.0131 (0.0151)  loss_rpn_box_reg: 0.0058 (0.0080)  time: 0.2758  data: 0.0122  max mem: 3913
Epoch: [1]  [20/51]  eta: 0:00:08  lr: 0.002000  loss: 0.1200 (0.1344)  loss_classifier: 0.0273 (0.0323)  loss_box_reg: 0.0792 (0.0841)  loss_objectness: 0.0100 (0.0124)  loss_rpn_box_reg: 0.0034 (0.0056)  time: 0.2644  data: 0.0003  max mem: 3913
Epoch: [1]  [30/51]  eta: 0:00:05  lr: 0.002000  loss: 0.1100 (0.1299)  loss_classifier: 0.0230 (0.0308)  loss_box_reg: 0.0731 (0.0828)  loss_objectness: 0.0081 (0.0110)  loss_rpn_box_reg: 0.0026 (0.0054)  time: 0.2647  data: 0.0003  max mem: 3913
Epoch: [

## Avalia Modelo

In [14]:
from my_utils.my_utils import evaluate_ap_multi_iou
import numpy as np

csv_path = "/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/results_fine_tuning_cmp.csv"
df_results = evaluate_ap_multi_iou(
    model=model,
    loader=cmp_watertanks_test_loader,
    device=device,
    score_thr=0.25,
    iou_list=(0.50, 0.75, 0.90),
    iou_grid=np.arange(0.50, 0.96, 0.05),
    out_csv_path=csv_path
)

==== Resultados por classe ====
Classe 1 | mAP[.50:.95]=0.505 | AP@0.50=0.898 | AP@0.75=0.487 | AP@0.90=0.022 (GT=50)

📊 CSV salvo em: /content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/results_fine_tuning_cmp.csv


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/model/wt_t_.pth')

In [16]:
from my_utils.my_utils import show_predictions

show_predictions(
    model=model,
    loader=watertanks_test_loader,
    device=device,
    n=2,
    score_thr=0.25
)


Output hidden; open in https://colab.research.google.com to view.